# Lab 4 - Pushover, OpenAI Agents SDK + TOOLS!!

In [2]:
from dotenv import load_dotenv
from agents import Agent, Runner, function_tool
from openai.types.responses import ResponseTextDeltaEvent
import gradio as gr
import os
import requests

load_dotenv(override=True)

True

In [3]:
pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_url = "https://api.pushover.net/1/messages.json"

In [4]:
def send_push_notification(message: str):
    payload = {"user": pushover_user, "token": pushover_token, "message": message}
    requests.post(pushover_url, data=payload)

In [5]:
#To receive Pushover notifications from e-mails, send to:

#9qaynk6gw8@pomail.net


In [6]:
send_push_notification("Hello, me!")

In [17]:
@function_tool
def push(age: int, message: str) -> str:
    """Send a text message as a push notification to Josh with this brief message

    Args:
        age: The age of the person
        message: The short text message to push to Josh 
    """

    send_push_notification(message)
    return "Push notification sent"

In [18]:
push.params_json_schema

{'properties': {'age': {'description': 'The age of the person',
   'title': 'Age',
   'type': 'integer'},
  'message': {'description': 'The short text message to push to Josh',
   'title': 'Message',
   'type': 'string'}},
 'required': ['age', 'message'],
 'title': 'push_args',
 'type': 'object',
 'additionalProperties': False}

In [13]:
instructions = """
You represent the AI Digital Twin of a human called Josh Janzen.
You are friendly and amiable, and you introduce yourself as Josh Janzen's Twin.
Josh is a leader of Data Science at Hormel Foods.
He loves coding and experimenting with LLMs, and he speaks at conferences and lectures about LLMs.
You chat with visitors on Josh's personal website. You answer questions about Josh's work.
If you don't know the answer, send a push notification to Josh to tell him the question you couldn't answer, so that he adds it to the knowledge base.
"""

In [14]:
agent = Agent(name="Twin", instructions=instructions, model="gpt-4.1-mini", tools=[push])

In [15]:
async def chat(message, history):
    messages = [{"role": prior["role"], "content": prior["content"]} for prior in history]  
    messages += [{"role": "user", "content": message}]
    response = Runner.run_streamed(agent, messages)
    reply = ""
    async for event in response.stream_events():
        if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
            reply += event.data.delta
            yield reply

In [16]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
